# Finetune ruGPT3Small on your data


## Install enviroment

In [27]:
!pip3 install transformers==4.19.3 -i https://pypi.org/simple

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 18.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2
  Attempting uninstall: transformers
    Found existing installation: transformers 2.8.0
    Uninstalling transformers-2.8.0:
      Successfully uninstalled transformers-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
taxi-pyml 0.1.0 requires torch==1.5.0, but you have torch 1.11.0 which is incompatible.
taxi-pyml 0.1.0 requires torchvision==0.6.0, but you have torchvision 0.12.0 which is incompatible.
taxi-pyml 0.1.0 requires transform

In [2]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2022-06-10 00:13:01--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0.01s   

2022-06-10 00:13:02 (3.28 MB/s) - ‘pretrain_transformers.py’ saved [34150/34150]



In [3]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2022-06-10 00:13:02--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0.004s  

2022-06-10 00:13:02 (2.52 MB/s) - ‘generate_transformers.py’ saved [10474/10474]



In [5]:
ls /usr/local

bin/   cuda-10.1/  games/    lib/  sbin/   src/  yabs-graphite-client/
cuda@  etc/        include/  man@  share/  www@


In [13]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [2]:
!nvidia-smi

Fri Jun 10 15:42:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   49C    P0    48W / 250W |  29306MiB / 32510MiB |     38%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:00:06.0 Off |                    0 |
| N/A   

In [17]:
!ps aux | grep 539007

bertan    539007 92.8  1.1 16922756 738840 ?     Rsl  00:01  28:19 /home/bertan/venv/mlenv/bin/python3.7 -m ipykernel_launcher -f /home/bertan/.local/share/jupyter/runtime/kernel-ba43e0a5-c31a-4874-9aa4-c1e415359115.json
gorodion  675174  0.0  0.0  17388  3104 pts/8    Ss+  00:31   0:00 /bin/sh -c ps aux | grep 539007
gorodion  675176  0.0  0.0  19080   944 pts/8    S+   00:31   0:00 grep 539007


In [14]:
!sh setup.sh

fatal: destination path 'apex' already exists and is not an empty directory.
Using pip 22.1.1 from /home/gorodion/go_venv/lib/python3.7/site-packages/pip (python 3.7)
Looking in indexes: https://pypi.yandex-team.ru/simple/
Processing ./apex
  Running command python setup.py egg_info


  torch.__version__  = 1.11.0+cu102


  running egg_info
  creating /place/vartmp/pip-pip-egg-info-9s3jy9gh/apex.egg-info
  writing /place/vartmp/pip-pip-egg-info-9s3jy9gh/apex.egg-info/PKG-INFO
  writing dependency_links to /place/vartmp/pip-pip-egg-info-9s3jy9gh/apex.egg-info/dependency_links.txt
  writing top-level names to /place/vartmp/pip-pip-egg-info-9s3jy9gh/apex.egg-info/top_level.txt
  writing manifest file '/place/vartmp/pip-pip-egg-info-9s3jy9gh/apex.egg-info/SOURCES.txt'
  writing manifest file '/place/vartmp/pip-pip-egg-info-9s3jy9gh/apex.egg-info/SOURCES.txt'
  /home/gorodion/apex/setup.py:121: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
    warnings.warn

## Add data to colab


## Prepare data

In [18]:
DATA_PATH = 'lenta_tech.csv'

In [21]:
import pandas as pd

train_path = 'train.txt'
with open(train_path, 'w') as f:
    f.write('\n'.join(pd.read_csv(DATA_PATH)['text'].values))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [3]:
%env CUDA_VISIBLE_DEVICES=0,1

env: CUDA_VISIBLE_DEVICES=0,1


In [4]:
import torch
torch.cuda.device_count()

2

In [2]:
from torch.cuda import amp

In [1]:
!python -m torch.distributed.launch --nproc_per_node=4 pretrain_transformers.py \
    --output_dir=news_model \
    --cache_dir=rugpt3 \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --per_gpu_train_batch_size 16 \
    --gradient_accumulation_steps 16 \
    --num_train_epochs 5 \
    --block_size 512 \
    --overwrite_output_dir \
    --fp16 \
    # --do_eval \
    # --eval_data_file=valid.txt \

/home/gorodion/go_venv/lib/python3.7/site-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
06/10/2022 15:53:05 - WARNING - __main__ -   Process rank: 1, device: cuda:1, n_gpu: 1, distributed training: True, 16-bits training: True
06/10/2022 15:53:05 - WARNING - __main__ -   Process rank: 3, device: cuda:3, n_gpu: 

## Check our model

In [2]:
%run generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=news_model \
    --k=5 \
    --p=0.95 \
    --length=100

06/10/2022 16:08:13 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=5, length=100, model_name_or_path='news_model', model_type='gpt2', n_gpu=4, no_cuda=False, num_return_sequences=1, p=0.95, padding_text='', prompt='', repetition_penalty=1.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')


Context >>>  тех ньюс


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ruGPT:
тех ньюсмейкеров. В своем микроблоге в Twitter пользователь опубликовал фотографию, на которой он держит в руках смартфон. По словам мужчины, это первый в его жизни гаджет, который он держит в руках. На снимке мужчина в черном, с трубкой. На снимке видно, что он держит смартфон в руках.
Американские ученые выяснили, как в человеческом организме могут изменяться химические реакции в процессе пищеварения. Результаты своих исследований авторы опубликовали в журнале Journal of the American Medical Association. Ученые выяснили, чт


Context >>>  вот ужее


...

Context >>>  Владимир Путин


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ruGPT:
Владимир Путин заявил, что не собирается отказываться от российского газа, если Россия откажется от его продажи. «Я бы не хотел, чтобы это произошло», — сказал Путин. Он отметил, что в России не было бы «Газпрома» и «Южного потока». Ранее президент России Владимир Путин заявил, что Россия не планирует отказываться от российского газа, если Россия откажется от него.
В сети появилось видео, на котором мужчина и женщина в масках пытаются найти спрятанную в шкафу бутылку с водой. Видео с кадро


Context >>>  


KeyboardInterrupt: 

## Some examples I got


Context >>> Is it

ruGPT:
Is it's the case that #Microsoft is using #SAP to manage #Microsoft’s business?

Context >>> It  is

ruGPT:
It is the only one that I've been trying to do.
Siri is on the verge of death, and I'm not aware of why she has to be on the verge.

Context >>> Why

ruGPT:
Why do I go here?  I'm wondering if I can use the #Microsoft Windows 8 Pro for the #WindowsPhone phone?  Thanks! 

In [3]:
from transformers import GPT2LMHeadModel

In [12]:
model = GPT2LMHeadModel.from_pretrained('news_model').to(DEVICE)

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
import torch
DEVICE = torch.device("cuda:0")

tokenizer = GPT2Tokenizer.from_pretrained('news_model', pad_token='')
# model = GPT2LMHeadModel.from_pretrained("best.pt", config='sberbank-ai/rugpt3small_based_on_gpt2').to(DEVICE)

AttributeError: 'GPT2Tokenizer' object has no attribute 'to'

In [112]:
help(GPT2Tokenizer.from_pretrained)

Help on method from_pretrained in module transformers.tokenization_utils_base:

from_pretrained(pretrained_model_name_or_path: Union[str, os.PathLike], *init_inputs, **kwargs) method of builtins.type instance
    Instantiate a [`~tokenization_utils_base.PreTrainedTokenizerBase`] (or a derived class) from a predefined
    tokenizer.
    
    Args:
        pretrained_model_name_or_path (`str` or `os.PathLike`):
            Can be either:
    
            - A string, the *model id* of a predefined tokenizer hosted inside a model repo on huggingface.co.
              Valid model ids can be located at the root-level, like `bert-base-uncased`, or namespaced under a
              user or organization name, like `dbmdz/bert-base-german-cased`.
            - A path to a *directory* containing vocabulary files required by the tokenizer, for instance saved
              using the [`~tokenization_utils_base.PreTrainedTokenizerBase.save_pretrained`] method, e.g.,
              `./my_model_directory

In [31]:
import pandas as pd
model.eval()

In [56]:
df = pd.read_csv('lenta_tech.csv')

In [ ]:
df = df.sample(frac=1)

In [ ]:
df.iloc[0]

'Житель американского штата Южная Каролина Омар Аван (Omar Awan) заживо сгорел в арендованном электромобиле Tesla после аварии. Об этом сообщает газета Washington Post. По данным издания, 48-летний мужчина потерял управление и врезался в пальму. В результате аварии литий-ионный аккумулятор автомобиля загорелся, огонь и дым заполнили салон. Вокруг собрались люди, однако им не удалось помочь мужчине из-за конструктивных особенностей модели S. Так, ручки машины утоплены в дверь и открываются с помощью электроники. В результате мужчина скончался от ожогов и удушения. Сторона обвинения настаивает на том, что причиной смерти стали конструктивные дефекты автомобиля. «Эта особенность Model S сделала автомобиль смертельной ловушкой», — говорится в иске семьи погибшего. В марте 2017 года два автомобиля Tesla Model S сгорели на территории зарядной станции Supercharger в Шанхае. Воспламенение произошло в одной из машин, после чего огонь перекинулся на другой электромобиль, припаркованный рядом.'

In [ ]:
df[df.text.str.split().str[0].apply(len) == 3].sample(10).text.values

array(['Под Стоунхенджем обнаружено гигантское ритуальное сооружение эпохи неолита, состоящее из камней высотой до четырех с половиной метров. Монолиты были найдены на глубине 90 сантиметров с помощью радаров. Археологи предполагают, что в древности они составляли не круг, как Стоунхендж, а ровную линию, и маркировали маршрут пути паломников. Об открытии сообщает The Telegraph. Находка была сделана на месте памятника Даррингтон Уоллс — обнесенного круглым валом поселения эпохи строительства Стоунхенджа. Еще с ХIX века ученых удивляло, что одна из сторон вала вытянута в ровную линию. Сейчас выяснилось, что этот край на самом деле составляет единое целое с рядом из 90 гигантских камней, имеющим форму подковы. Скорее всего, эти мегалиты обозначали путь паломников к Стоунхенджу или другому ритуальному сооружению. «Это одно из крупнейших мегалитических сооружений в Европе, которое скрывалось у нас под носом четыре тысячи лет. Мы пока не знаем, каков минеральный состав этих камней. Если по в

In [91]:
rand_text = 'Thsssse Tor Project'.split()
if len(rand_text[0]) <= 6:
    prompt = ' '.join(rand_text[:3])
else:
    prompt = rand_text[0]

In [92]:
prompt

'Thsssse'

In [113]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [125]:
out = model.generate( 
            input_ids,
            do_sample=True,
            num_beams=2,
            temperature=1.5,
            top_p=0.9,
            max_length=20,
            )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [153]:
from tqdm.notebook import trange

# generated = []
with open('generated2.txt', 'w') as f:
    for i in trange(15000):
        rand_text = df.iloc[i].text.split()
        if len(rand_text[0]) <= 6:
            prompt = ' '.join(rand_text[:3])
        else:
            prompt = rand_text[0]
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate( 
                                input_ids,
                                do_sample=True,
                                num_beams=2,
                                temperature=1.5,
                                top_p=0.9,
                                max_length=100,
                                pad_token_id=50256
                                )

        generated_text = list(map(tokenizer.decode, out))[0]
        print(generated_text, end='<EOS>\n', file=f)

  0%|          | 0/15000 [00:00<?, ?it/s]

In [151]:
generated_text

'Житель американского штата Вашингтон и его друзья обнаружили у себя в доме три трупа животных. Об открытии сообщается в пресс-релизе на EurekAlert!. В квартире были обнаружены пять человеческих тел: один лежал лицом вниз, другой — лежа на полу, а третий лежал на боку. В квартире были обнаружены три трупа животных, все они принадлежали людям. Всего за несколько месяцев ученые зафиксировали семь новых случаев заражения животных бешенством, а также обнаружили пять случаев смерти от других инфекций. По данным издания,'

In [142]:
pd.Series(generated).to_csv('generated.txt', index=False, header=False)

In [147]:
!head -2 generated.txt

"Житель американского штата Небраска (штат Невада, США) обнаружил в своем Instagram фотографию, на которой запечатлена женщина со своим ребенком.
Пользователи социальных сетей поделились фотографиями с ребенком, который сидит на качелях. В комментариях под роликом пользователи выложили видео, на котором изображена младшая дочь ребенка, находящегося на качелях в возрасте 1,5 месяца. В комментариях пользователи также рассказали, что ребенок сидит на них в течение полутора месяцев. На видео видно, что"
